# Ray Surveyor results analysis - Microbiome

## Import modules and ray surveyor scripts

In [ ]:
# Import modules 
import numpy as np
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import plotly.offline as py

# raysurveyor scripts
from surveyor_scripts.raysurveyor_gentree import read_matrix, build_tree, plotly_heatmap
from surveyor_scripts.matrix_transform import drop_indices, normalize_gram_matrix 
from surveyor_scripts.treeclust_compare import cophentic_correlation_test, mantel_test

# plotly notebook mode
py.init_notebook_mode()

## Generate a phenetic tree from the global kmer Gram matrix

* read the matrix and get rid of the indices linked to the filtering datasets
* build a neighbor-joining tree from the derived distance matrix
* build a dendrogram / heatmap of the distance matrix

In [ ]:
# Read Gram matrix as a pandas dataframe
sim_matrix_global = read_matrix('./survey.res.microbiomes/Surveyor/SimilarityMatrix.global.tsv')

# Remove filtering dataset entries from the global Gram matrix to keep only the genome entries
# print(sim_matrix_global.axes[0])
sim_matrix_global = drop_indices(sim_matrix_global, [0,1,2,3,4,5,6])
# display(sim_matrix_global)

In [ ]:
# Neighbor-Joining tree from the gram matrix -> distance matrix

# Get the normalized matrix
norm_mat = normalize_gram_matrix(sim_matrix_global.as_matrix())

# Generate a distance matrix from normalized matrix with a distance function (euclidean)
dist_mat = squareform(pdist(norm_mat, 'euclidean'))
df_dist = pd.DataFrame(dist_mat, index=sim_matrix_global.index, columns=sim_matrix_global.columns)
# display(df_dist)

# Build the nj tree with ete3
names_list = [str(i) for i in sim_matrix_global.index.values.tolist()]
tree = build_tree(df_dist, names_list, 'nj')
tree.render("%%inline", w=500)

## Questions :
### What can you deduce from the tree in terms of patient microbiome diversity VS the antibiotic effect ?
### Patients 6,7,8,23,25,38 are controls is there some indication of that in the tree ? 

In [ ]:
# Plot a dendrogram and heatmap for whole genome clusters
figure = plotly_heatmap(df_dist, title="Whole Genomes", width=900, height=900)
py.iplot(figure)

## Create hierarchical cluster dendrogram for the filtered matrices (Insertion Sequences, Resistance Genes, Phages)


### * Insertion Sequences Filtering

In [ ]:
# Load and display the filtered datasets
sim_matrix_is_filter = read_matrix('./survey.res.microbiomes/Surveyor/SimilarityMatrix.filter-1.tsv')
# print(sim_matrix_is_filter.axes[0])
sim_matrix_is_filter = drop_indices(sim_matrix_is_filter, [0])
# display(sim_matrix_is_filter)

# Plot a dendrogram and heatmap
dist_matrix_is_filter= pd.DataFrame(squareform(pdist(normalize_gram_matrix(sim_matrix_is_filter.as_matrix()), 'euclidean')), index=sim_matrix_global.index, columns=sim_matrix_global.columns)
figure = plotly_heatmap(dist_matrix_is_filter, title="IS - Genomes Filtering", width=900, height=900)
py.iplot(figure)


### * Antibiotic Resistance Genes Filtering

In [ ]:
# Load and display the filtered datasets
sim_matrix_ar_filter = read_matrix('./survey.res.microbiomes/Surveyor/SimilarityMatrix.filter-2.tsv')
# print(sim_matrix_ar_filter.axes[0])
sim_matrix_ar_filter = drop_indices(sim_matrix_ar_filter, [0])
# display(sim_matrix_ar_filter)

# Plot a dendrogram and heatmap
dist_matrix_ar_filter= pd.DataFrame(squareform(pdist(normalize_gram_matrix(sim_matrix_ar_filter.as_matrix()), 'euclidean')), index=sim_matrix_global.index, columns=sim_matrix_global.columns)
figure = plotly_heatmap(dist_matrix_ar_filter, title="Antibiotic Resistance - Genomes Filtering", width=900, height=900)
py.iplot(figure)

## * Bacteriophages Filtering (to complete)

* Take example from the filtered matrices 1 and 2 below
* Use file ./survey.res.microbiomes/Surveyor/SimilarityMatrix.filter-3.tsv for the matrix file
* Name your you distance matrix dist_matrix_phages_filter


In [ ]:
# PUT YOUR CODE HERE for the Bacteriophages Filtering


## Compare whole genome and filtered genome clustering

In order to evaluate how the filtered genome clusters correlate with the whole genome cluster, we can use the cophenetic correlation coefficient (CCC).
You can read more about Cophenetic correlation here : https://en.wikipedia.org/wiki/Cophenetic_correlation.

Essentially, the coefficient gives a confidence of how well the distance between the leafs of a dendrograms are preserve with unmodelled datapoints.
When used with the whole genome cluster as the reference datapoints it gives us an index of similarities between clusters.

The CCC is a value between 0-1; 0 meaning that there's no correlation at all between the 2 clustering and 1 means a perfect cluster's correlation.


In [ ]:
# Print Cophenetic Correlation Coefficient
print("CCC between Whole-Genome VS IS Filtering : CCC %s, p-value %s" % cophentic_correlation_test(df_dist,dist_matrix_is_filter))
print("CCC between Whole-Genome VS AR Filtering : CCC %s, p-value %s" % cophentic_correlation_test(df_dist,dist_matrix_ar_filter))
print("CCC between Whole-Genome VS Phages Filtering : CCC %s, p-value %s" % cophentic_correlation_test(df_dist,dist_matrix_phages_filter))

## Question
### Which filtered dataset correlate the best with the whole genome clusters ?